# CRAGIG RUN 1
20170202 

I got my first batch of RAD data back from BGI, and have yet to really look at it. I'd like to take a first look for population differences, but also I need to help Molly figure out how to build a parentage panel. My first look will be at the **forward reads.**

### Download Data from BGI

I downloaded the data by clicking on the download FTP button. Downloading each of the two raw data files took about 24 hours, and crashed occasionally. I then moved the two files onto the E drive (which has a ton of space), copied them, and saved on as untouchable. The other, I put in a shared folder so I can play with it in my virtual machine.

### Check that Download was Error Free

used md5sum, a built in function in Ubuntu, and compared the check sum associated with the raw forward read file and it matched

```
nclowell@ubuntu:/mnt/hgfs/Data_for_Analysis/WorkingFolder$ md5sum 161228_I137_FCHCYV5BBXX_L5_CHKPE85216120009_1.fq.gz

6d7714b6f03d65002894ba5d1c16b520  161228_I137_FCHCYV5BBXX_L5_CHKPE85216120009_1.fq.gz
```

While I'm' at it, going to check the reverse reads too:

```
nclowell@ubuntu:/mnt/hgfs/Data_for_Analysis/Raw_data$ md5sum 161228_I137_FCHCYV5BBXX_L5_CHKPE85216120009_2.fq.gz 
27ed5a022a51bbc5fa23300bb22e602e  161228_I137_FCHCYV5BBXX_L5_CHKPE85216120009_2.fq.gz
```

### ``process_radtags``

``process_radtags`` dumultiplexes your raw data file and (optionally) names the sample by file name. Be sure to include unique file names for any replicates. For example, fg100-fg104, I sequenced twice, each with a unique barcode.


[mary says don't use filterillumina; used r c q and trimmed]

Stacks [manual page for ``process_radtags``](http://catchenlab.life.illinois.edu/stacks/comp/process_radtags.php)

Inputs values I will use

 f — path to the input file if processing single-end seqeunces 
<br>**-f /mnt/hgfs/Data_for Analysis/WorkingFolder/161228_I137_FCHCYV5BBXX_L5_CHKPE85216120009_1.fq.gz**

 i — input file type, either 'bustard' for the Illumina BUSTARD format, 'bam', 'fastq' (default), or 'gzfastq' for gzipped FASTQ.
<br>**-i gzfastq**

 y — output type, either 'fastq', 'gzfastq', 'fasta', or 'gzfasta' (default is to match the input file type).
<br>**-y gzfastq**

 o — path to output the processed files.
<br>**-o /mnt/hgfs/Data_for Analysis/WorkingFolder/Stacks**

 t — truncate final read length to this value.
<br>**-t 139** (reads are 150 bp, 6 of which are barcode, and trim at least 5, so start with 139. Then run FastQC and see if more trimming is necessary)

 b — path to a file containing barcodes for this run.
<br>**-b /mnt/hgfs/Data_for Analysis/WorkingFolder/barcodes_cragigrun1.txt**

 r — rescue barcodes and RAD-Tags (this means if barcode is one base off from any barcode you used, Stacks will use some probability function to match to closest barcode and rewrite sequence to reflect that)
<br>**-r**

 c — clean data, remove any read with an uncalled base.
<br>**-c**

 -e [enz], --renz_1 [enz]: provide the restriction enzyme used (cut site occurs on single-end read) <- 'sbfI'
<br>**-e sbfI**

Your barcode list:

Should take the form of barcode \t samplename \n

Local path for my current barcode list = 

Change to appropriate working directory

In [2]:
cd /mnt/hgfs/Data_for Analysis/WorkingFolder

/mnt/hgfs/Data_for Analysis/WorkingFolder


In [ ]:
!stacks process_radtags \
-f /mnt/hgfs/Data_for_Analysis/WorkingFolder/161228_I137_FCHCYV5BBXX_L5_CHKPE85216120009_1.fq.gz \
-i gzfastq \
-y gzfastq \
-o /mnt/hgfs/Data_for_Analysis/WorkingFolder/Stacks \
-t 139 \
-b /mnt/hgfs/Data_for_Analysis/WorkingFolder/barcodes_cragigrun1.txt \
-r \
-c \
-e sbfI

**``process_radtags`` output:**

387289855 total sequences;
  64147742 ambiguous barcode drops;
  401015 low quality read drops;
  42895231 ambiguous RAD-Tag drops;
279845867 retained reads.

So 72.26% reads retained.

Then, I ran a script to count reads in each post-``process_radtags`` file, which Steven gave to Mary who gave to me. Here's the script. A similar version is on her github, [here](https://github.com/mfisher5/mf-fish546-PCod/blob/master/scripts/L1L2_seqCountsgen.py).

```
import sys
lane1 = open(sys.argv[1], "r")
newshell = open("CountFASTQseqs.sh", "w")

newshell.write("#!/bin/bash" + "\n\n")

for line in lane1:
	linelist = line.strip().split()
	filestring = "zcat L1L2samples/" + linelist[0] + ".fq.gz | awk '((NR-2)%4==0){read=$1;total++;count[read]++}END{for(read in count){if(!max||count[read]>max) {max=count[read];maxRead=read};if(count[read]==1){unique++}};print total,unique,unique*100/total,maxRead,count[maxRead],count[maxRead]*100/total}' >> FastQsequenceCounts.txt"
	newshell.write(filestring + "\n")
lane1.close()
newshell.close()
```

I'm still working on getting ``matplotlib`` installed in my virtual machine, but until then, I exported the results to excel, sorted, and picked the ten samples with the most reads out of each population. For Alaska

## Quality check with FastQC

Looked at several individual sample FastQC reports and they all look great. Mean average quality score of 40, with little variation, and none dropped below the green.

## ``ustacks``

I used a python script I wrote previously to build the shell script. There was an error where it skipped the first few of my samples and didn't run ``ustacks`` on them, so I need to improve it. Also it would be great to be able to input the different parameters with flags. And my brother showed me a module that helps you do that... so I'm going to try to do that now.

Example command to show parameter values:

``stacks ustacks -t gzfastq -f ./Stacks/Q334.fq.gz -r -d -o ./Stacks -i 001 -m 10 -M 3 -p 10``


Only one sample retained few loci (FG100_A retained only 6) so I'm going to exclude that one from further analyses. All of the rest of the samples had at least 20K, with the vast majority around 25K-30K.

## Pick samples for ``cstacks``

I just got clarified from lab mates that we're supposed to use the 10 most sequenced individuals from **each** population. I don't even have 10 from each in this run, so I'll include up to 10 from each population.

Sorting my Excel sheet to determine these...

These leads me to:
Q324
Q318
Q326
Q314
Q316
Q323
Q325
Q351
Q352
Q332
Q339
Q334
Q347
Q356
Q330
Q355
Q354
FG009
FG033
FG002
FG027
FG021
FG013
FG001
FG005
FG014
FG006
FG101_A
FG102_A
FG100_B
FG106
FG107
FG109
FG104_A
FG111
FG108
FG103_A
FG210
FG205
FG211
FG212
FG201
FG209
FG206
FG204
FG202
FG203


<br><br> There are 47 because the AK population only had 7


## ``cstacks``


Now I need to write the ``cstacks`` shell script to run the program on these ten samples. I used my ``easy_cstacks.py`` script.

Code for ``cstacks`` was:

```
stacks cstacks -b 100 -s Stacks/Q324 -s Stacks/Q318 -s Stacks/Q326 -s Stacks/Q314 -s Stacks/Q316 -s Stacks/Q323 -s Stacks/Q325 -s Stacks/Q351 -s Stacks/Q352 -s Stacks/Q332 -s Stacks/Q339 -s Stacks/Q334 -s Stacks/Q347 -s Stacks/Q356 -s Stacks/Q330 -s Stacks/Q355 -s Stacks/Q354 -s Stacks/FG009 -s Stacks/FG033 -s Stacks/FG002 -s Stacks/FG027 -s Stacks/FG021 -s Stacks/FG013 -s Stacks/FG001 -s Stacks/FG005 -s Stacks/FG014 -s Stacks/FG006 -s Stacks/FG101_A -s Stacks/FG102_A -s Stacks/FG100_B -s Stacks/FG106 -s Stacks/FG107 -s Stacks/FG109 -s Stacks/FG104_A -s Stacks/FG111 -s Stacks/FG108 -s Stacks/FG103_A -s Stacks/FG210 -s Stacks/FG205 -s Stacks/FG211 -s Stacks/FG212 -s Stacks/FG201 -s Stacks/FG209 -s Stacks/FG206 -s Stacks/FG204 -s Stacks/FG202 -s Stacks/FG203 -o Stacks -n 3 -p 3 

```
Worked! And it took ~6.5 hours to run.

```
Writing catalog to 'Stacks/... done.

Running cstacks took 
06:27:18.40
```



## ``sstacks``

Need to get all the way through populations in order to form catalog of loci for Bowtie and Blast filtering that will then feed back into the Stacks pipeline starting at ``pstacks``.

I wrote a script that makes the ``sstacks`` shell, and runs it. Going to update it and run it. 

The shell script looks like this:

```
stacks sstacks -b 100 -s Stacks/Q351 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q352 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q332 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q339 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q334 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q347 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q356 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q330 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q355 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q354 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q338 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q353 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q324 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q318 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q326 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q314 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q316 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q323 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/Q325 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG009 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG033 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG002 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG027 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG021 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG013 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG001 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG005 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG014 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG006 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG004 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG024 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG017 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG003 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG018 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG019 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG016 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG015 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG031 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG032 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG007 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG026 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG011 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG035 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG034 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG010 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG101_A -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG102_A -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG100_B -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG106 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG101_B -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG107 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG109 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG102_B -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG104_A -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG111 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG108 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG103_A -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG103_B -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG104_B -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG100_A -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG210 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG205 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG211 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG212 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG201 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG209 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG206 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG204 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG202 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG203 -o Stacks/ -p 3 -c Stacks/batch_100 
stacks sstacks -b 100 -s Stacks/FG207 -o Stacks/ -p 3 -c Stacks/batch_100 

```

It worked! And it took 00:13:10.65 to run.



## Populations

#### Question: 
How should my parameter sets compare between the first round of ``populations`` (to build reference genome) and the second round of ``populations`` when I'm actually comparing populations for popgen?

Just talked to Dan, who recommended that I be pretty lenient the first time through, and then very stringent (for Molly's parentage panel needs) the second time through. For lenient, he said an r of 0.25 and for p, about half of the number of populations.

Code for this run:

```
stacks populations -b 100 -P Stacks -M popmap_cragigrun1.txt -t 10 -r 0.25 -p 3 -m 5 --genepop
```

<br>Here:
<br> -b 100 (batch num)
<br> -P Stacks (directory w sstacks output files)
<br> -M popmap_cragigrun1.txt (pop map)
<br> -t 10 (threads)
<br> -r .25 (percentage ind in pop required to process locus for pop)
<br> -p 3 (min populations loci present in to keep locus)
<br> -m 5 specify a minimum stack depth required for individuals at a locus.

Output
```
Writing 28647 loci to summary statistics file, 'Stacks/batch_100.sumstats.tsv'
Writing 28647 loci to observed haplotype file, 'Stacks/batch_100.haplotypes.tsv'
Writing population data to GenePop file 'Stacks/batch_100.genepop'
```

Counted loci by columns in pop gen file - 1 (for first column of sample names):

```
nclowell@ubuntu:/mnt/hgfs/Data_for_Analysis/WorkingFolder$ python count_loci_genepop.py Stacks/batch_100.genepop
Your genepop file has 232763 columns of genotypes.
```



## Bowtie + BLAST filtering

6. Filtering loci with bowtie + BLAST

We filter out highly repetive loci using bowtie and BLAST. More information in this markdown file. 

#### A. Filtering with bowtie
Make a fasta file for bowtie by running a custom python script that Mary wrote, which requires (1) a text file with the header from the genepop file and (2) unzipped batch.catalog.tags file.

Here, I will make batch_100_loci.txt for this round by manually opening the genepop file in Textwrangler, and cutting and pasting the header line with the tag names into its own text file.

first bit of the file:

In [8]:
!python get_first_line_genepop.py batch_100 batch_100_loci_names.txt

3_11,3_19,3_28,3_38,3_50,3_103,3_130,4_7,4_14,4_39,4_41,4_44,4_46,4_58,4_59,4_61,4_62,4_69,4_76,4_77,4_78,4_85,4_90,4_133,7_20,7_105,7_130,7_131,9_20,9_26,9_36,9_53,9_67,9_70,9_79,9_101,9_117,9_119,9_125,9_137,9_138,10_32,11_96,11_131,12_32,12_50,12_73,12_96,12_98,12_117,12_138,14_8,14_15,14_23,14_29,14_38,14_43,14_44,14_49,14_65,14_67,14_107,16_16,16_46,16_50,16_54,16_73,16_77,16_79,16_97,16_108,16_109,17_29,17_30,17_37,17_48,17_50,17_85,17_89,17_97,17_108,17_113,17_120,18_9,18_13,18_35,18_46,18_53,18_59,18_65,18_70,18_71,18_90,18_91,18_95,18_97,18_99,18_108,18_112,18_114,18_117,18_118,18_119,18_136,23_58,23_118,23_119,23_120,26_49,26_55,26_89,26_90,26_137,27_8,27_28,27_49,27_67,27_76,27_86,27_134,27_135,32_22,32_23,32_35,32_39,32_52,32_75,32_84,32_101,32_120,32_121,32_136,32_137,33_9,33_13,33_33,33_52,33_127,36_16,36_18,36_33,36_45,36_58,36_94,36_116,36_126,36_133,37_10,37_23,37_34,37_67,37_82,37_85,37_109,37_111,37_117,37_118,38_18,38_29,38_32,38_50,43_8,43_21,43_27,43_30,43_32,43_3

Then I will unzip the catalog tags file with ``gzip -d batch_100.catalog.tags.tsv.gz``

In [9]:
cd Stacks

/mnt/hgfs/Data_for_Analysis/WorkingFolder/Stacks


In [11]:
!gzip -d batch_100.catalog.tags.tsv.gz

And now Mary's script...

In [14]:
!python gen_bowtie_fasta.py batch_100_loci_names.txt Stacks/batch_100.catalog.tags.tsv

Installed bowtie by downloading the zipped folder, navigating to that folder, and running

```
nclowell@ubuntu:/mnt/hgfs/Data_for_Analysis/WorkingFolder/Bowtie/bowtie-1.2$ sudo apt install bowtie
[sudo] password for nclowell: 
```

Manually made a new folder to hold the bowtie software and files I make with bowtie.

Now, make bowtie index:

In [26]:
!bowtie-build seqsforBOWTIE.fa batch_100

Settings:
  Output files: "batch_100.*.ebwt"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 5 (one in 32)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  seqsforBOWTIE.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 954860
Using parameters --bmax 716145 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 716145 --dcv 1024
Constructing suffix-array element generator
Buildi

Now align sequenecs to the index you just made to identify any sequences that align to multiple places in the genome (maybe highly repetive loci like microsats).

In [28]:
!bowtie -f -v 3 --sam --sam-nohead \
batch_100 \
seqsforBOWTIE.fa \
batch_100_BOWTIEout.sam

# reads processed: 27478
# reads with at least one reported alignment: 27478 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 27478 alignments to 1 output stream(s)


Then use Dan's custom script to remove unwanted sequences after bowtie alignment.

In [30]:
!python parseBowtie_DD.py batch_100_BOWTIEout.sam batch_100_BOWTIEout_filtered.fa

Number of Bowtie output lines read: 27478 5089t lines read: 8637d: 14625 lines read: 17738owtie output lines read: 20232 lines read: 22858umber of Bowtie output lines read: 25545
Number of sequences written to output: 27476ritten to output: 2655written to output: 646nces written to output: 789mber of sequences written to output: 956itten to output: 1166 output: 1352 of sequences written to output: 1562mber of sequences written to output: 1725sequences written to output: 1911s written to output: 2097 2306mber of sequences written to output: 2493sequences written to output: 2679s written to output: 2865n to output: 3051itten to output: 3214 output: 3400n to output: 3563mber of sequences written to output: 3773s written to output: 3889mber of sequences written to output: 40291 of sequences written to output: 4378n to output: 4587mber of sequences written to output: 4797mber of sequences written to output: 5053itten to output: 52621 output: 5704 of sequences written to output: 5914 of sequ

I ran this command in the terminal outside jupyter notebook because the auto updating of the sys.stout.write() was freakin' out the notebook.

```
python parseBowtie_DD.py batch_100_BOWTIEout.sam batch_100_BOWTIEout_filtered.fa
```

Output:
```
Number of Bowtie output lines read: 27478
Number of sequences written to output: 27476
```

To see how many loci were filtered out due to Bowtie, first count how many unique loci were in the genepop file.

In [33]:
!python count_loci_genepop.py Stacks/batch_100.genepop

Your genepop file has 232763 columns of genotypes.


I think that means Bowtie filtered out a ton of loci. 232763-27478 = 205285 = 88%

That's sort of insane compared to when I ran through the Pacific cod data, but I did use extremely lenient parameters on this run of populations.

## Filtering with Blast

Make a blast database with filtered output from Bowtie filtering, then blast sequences to database and remove any loci that match other loci equally well or better than to themselves. This is supposed to remove highly repetitive loci like microsatellites that can interfere with our data analysis.

Need to install Blast, make sure I can run it from anywhere, make a directory for Blast files, move the filtered bowtie fasta there, then blast sequences to the database and filter with Dan's Blast parsing script.

First step... install Blast on Ubuntu:

```
nclowell@ubuntu:/mnt/hgfs/Data_for_Analysis/WorkingFolder$ sudo apt-get install ncbi-blast+
```

Make sure blast is working:
```
nclowell@ubuntu:/mnt/hgfs/Data_for_Analysis/WorkingFolder$ blastp
BLAST query/options error: Either a BLAST database or subject sequence(s) must be specified
Please refer to the BLAST+ user manual.
```
Now, make a directory for Blast files.

<img src="bowtie_folder_made.png">

In [37]:
!makeblastdb -in batch_100_BOWTIEout_filtered.fa \
-parse_seqids \
-dbtype nucl \
-out batch_100_BOWTIEfiltered




Building a new DB, current time: 02/09/2017 11:04:48
New DB name:   /mnt/hgfs/Data_for_Analysis/WorkingFolder/Blast/batch_100_BOWTIEfiltered
New DB title:  batch_100_BOWTIEout_filtered.fa
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 27476 sequences in 0.571891 seconds.


Now blast against itself:

In [38]:
!blastn -query batch_100_BOWTIEout_filtered.fa \
-db batch_100_BOWTIEfiltered \
-out batch_100_BowtieBlastFiltered

Use Dan's python script to filter only wanted loci:

It was crashing the notebook... in the terminal I ran:
```
nclowell@ubuntu:/mnt/hgfs/Data_for_Analysis/WorkingFolder/Blast$ python checkBlastResults_DD.py \
> batch_100_BowtieBlastFiltered \
> batch_100_BOWTIEout_filtered.fa \
> batch_100_BowtieBlastFiltered_GOOD.fa \
> batch_100_BowtieBlastFiltered_BAD.fa

Identifying which loci are 'good' and 'bad' based on BLAST alignments...
Writing 'good' and 'bad' loci to their respective files...
```

Now, check how many loci were filtered out during the BLAST phase of filtering.

In [3]:
cd Data_for_Analysis/WorkingFolder/Blast/

/mnt/hgfs/Data_for_Analysis/WorkingFolder/Blast


In [41]:
!grep ">" batch_100_BowtieBlastFiltered_GOOD.fa | wc -l

27460


In [4]:
!grep ">" batch_100_BowtieBlastFiltered_BAD.fa | wc -l

16


So that makes 27460 - 16 = 27444, or <.1%.

## Creating final reference genome with Bowtie

Lastly, I need to use Bowtie again to build a final Bowtie index using the files cleaned in Blast, and then use Bowtie to align all of my fastq files to the Bowtie index for pstacks. So I manually need to move the output files from Blast to the Bowtie folder. Its these alignments (of each fastq with the final reference genome) that feed into ``pstacks``.

Ran this outside the notebook because it crashes the notebook:

In [ ]:
!bowtie-build batch_100_BowtieBlastFiltered_GOOD.fa batch_100_final_index

Now, need to align each fastq file against final reference genome. I made a Python script to write that bash script. Going to see if it needs updating...

Sample bash script line:
``bowtie -q -v 3 --norc --sam Blast/batch_100_final_index Stacks/Q351.fq Q351.sam``

Output:

...
```
Reported 2500668 alignments to 1 output stream(s)
# reads processed: 3153859
# reads with at least one reported alignment: 2269973 (71.97%)
# reads that failed to align: 883886 (28.03%)
Reported 2269973 alignments to 1 output stream(s)

Running bowtie took 
08:10:31.52
```

Now into ``pstacks``!

## ``pstacks``

Wrote a ``pstacks`` bash with a script I made last quarter.

Sample ``pstacks`` line of code:

``stacks pstacks -p 5 -m 10 -t sam -i 1 -f Stacks/Q351.sam -o Stacks``

Output:

...
```
Parsing Stacks/FG207.sam
  Analyzed 2269973 sequence reads; Identified 146488 unique stacks from those reads.
  Merged 149407 unique Stacks into 17980 loci.
Identifying polymorphic sites and calling consensus sequences...done.
  Number of utilized reads 2269973
  Mean coverage depth is 129.1; Std Dev: 243.874; Max: 14096
Writing loci, SNPs, alleles to 'Stacks/...'
  Wrote 17573 loci, excluded 407 loci due to insuffient depth of coverage; blacklisted 0 loci.
Finished running pstacks_shell.txt script.

Running pstacks took 
01:23:37.17

```

## Second round ``cstacks``

I'm going to use my ``easy_cstacks.py`` script. Notice that now we have to tags, snps, and alleles file, one set from ustacks and one now from pstacks. I moved all the pstacks ones into their own directory (called pstacks).

This is what the bash script looks like:

```
Currently, this program thinks you intend to include 47 samples in cstacks.

Type YES if correct. Type NO if incorrect and check your files and code.YES

Sample number verified. Program continuing.

Finished writing cstacks shell script.

This is what your cstacks shell looks like.
stacks cstacks -b 100 -g -s pstacks/Q324 -s pstacks/Q318 -s pstacks/Q326 -s pstacks/Q314 -s pstacks/Q316 -s pstacks/Q323 -s pstacks/Q325 -s pstacks/Q351 -s pstacks/Q352 -s pstacks/Q332 -s pstacks/Q339 -s pstacks/Q334 -s pstacks/Q347 -s pstacks/Q356 -s pstacks/Q330 -s pstacks/Q355 -s pstacks/Q354 -s pstacks/FG009 -s pstacks/FG033 -s pstacks/FG002 -s pstacks/FG027 -s pstacks/FG021 -s pstacks/FG013 -s pstacks/FG001 -s pstacks/FG005 -s pstacks/FG014 -s pstacks/FG006 -s pstacks/FG101_A -s pstacks/FG102_A -s pstacks/FG100_B -s pstacks/FG106 -s pstacks/FG107 -s pstacks/FG109 -s pstacks/FG104_A -s pstacks/FG111 -s pstacks/FG108 -s pstacks/FG103_A -s pstacks/FG210 -s pstacks/FG205 -s pstacks/FG211 -s pstacks/FG212 -s pstacks/FG201 -s pstacks/FG209 -s pstacks/FG206 -s pstacks/FG204 -s pstacks/FG202 -s pstacks/FG203 -o pstacks -n 3 -p 5 
Run this shell script?YES
Running shell script.
Number of mismatches allowed between stacks: 3
Loci matched based on genomic location.
Constructing catalog from 47 samples.
Initializing new catalog...
  Parsing pstacks/Q324.tags.tsv
  Parsing pstacks/Q324.snps.tsv
  Parsing pstacks/Q324.alleles.tsv
Building an index of the catalog.
Processing sample pstacks/Q324 [2 of 47]
  Parsing pstacks/Q318.tags.tsv
  Parsing pstacks/Q318.snps.tsv
  Parsing pstacks/Q318.alleles.tsv
Searching for matches by genomic location...
Merging matches into catalog...
  0 loci matched more than one catalog locus and were excluded.
  Updating catalog index...
Processing sample pstacks/Q318 [3 of 47]
  Parsing pstacks/Q326.tags.tsv

```

The output looks like:

...
```
Processing sample pstacks/FG204 [46 of 47]
  Parsing pstacks/FG202.tags.tsv
  Parsing pstacks/FG202.snps.tsv
  Parsing pstacks/FG202.alleles.tsv
Searching for matches by genomic location...
Merging matches into catalog...
  0 loci matched more than one catalog locus and were excluded.
  Updating catalog index...
Processing sample pstacks/FG202 [47 of 47]
  Parsing pstacks/FG203.tags.tsv
  Parsing pstacks/FG203.snps.tsv
  Parsing pstacks/FG203.alleles.tsv
Searching for matches by genomic location...
Merging matches into catalog...
  0 loci matched more than one catalog locus and were excluded.
  Updating catalog index...
Writing catalog to 'pstacks/... done.

Running cstacks took 
00:15:31.07

```
It ran so fast though - going to double-check that nothing is fishy.

In [8]:
!grep --count --with-filename consensus batch_100.catalog.tags.tsv

batch_100.catalog.tags.tsv:27460


Looks like I have 27460 loci in my catalog, that looks about normal (Eleni had 29K loci in her catalog).

## Second ``sstacks``

First line of the bash script looked like:

```
stacks sstacks -g -b 100 -s pstacks/Q351 -o pstacks -p 3 -c pstacks/batch_100
```

Output looked like:

...
```
Processing sample 'pstacks/FG207' [1 of 1]
  Parsing pstacks/FG207.tags.tsv
  Parsing pstacks/FG207.snps.tsv
  Parsing pstacks/FG207.alleles.tsv
Searching for matches by genomic location...
  Creating map of genomic locations...done.
17573 stacks matched against the catalog containing 27460 loci.
  17573 matching loci, 1527 contained no verified haplotypes.
  1204 loci contained SNPs unaccounted for in the catalog and were excluded.
  23178 total haplotypes examined from matching loci, 19817 verified.
Outputing to file pstacks/FG207.matches.tsv
00:24:00.03

```



## Second ``populations``

``
stacks populations -b 100 -P Stacks -M popmap_cragigrun1.txt -t 10 -r 1 -p 5 -m 5 --genepop
``


In [11]:
pwd

u'/mnt/hgfs/Data_for_Analysis/WorkingFolder'

In [13]:
!stacks populations -b 100 -P pstacks -M popmap_cragigrun1.txt -t 10 -r 1 -p 5 -m 10 --genepop

Fst kernel smoothing: off
Bootstrap resampling: off
Percent samples limit per population: 1
Locus Population limit: 5
Minimum stack depth: 10
Log liklihood filtering: off; threshold: 0
Minor allele frequency cutoff: 0
Maximum observed heterozygosity cutoff: 1
Applying Fst correction: none.
Parsing population map.
Found 71 input file(s).
  5 populations found
    WA_Strait: FG001, FG002, FG003, FG004, FG005, FG006, FG007, FG009, FG010, FG011, FG013, FG014, FG015, FG016, FG017, FG018, FG019, FG021, FG024, FG026, FG027, FG031, FG032, FG033, FG034, FG035
    WA_SanJuans: FG100_A, FG100_B, FG101_A, FG101_B, FG102_A, FG102_B, FG103_A, FG103_B, FG104_A, FG104_B, FG106, FG107, FG108, FG109, FG111
    WA_Dabob: FG201, FG202, FG203, FG204, FG205, FG206, FG207, FG209, FG210, FG211, FG212
    AK: Q314, Q316, Q318, Q323, Q324, Q325, Q326
    CA_Catalina: Q330, Q332, Q334, Q338, Q339, Q347, Q351, Q352, Q353, Q354, Q355, Q356
  1 group of populations found
    1: WA_Strait, WA_SanJuans, WA_Dabob, AK,

In [14]:
!python count_loci_genepop.py pstacks/batch_100.genepop

Your genepop file has 5027 columns of genotypes.


## Filtering shenanigans

Everyone in our lab uses a whole bunch of filtering scripts that I haven't used before, so I'll be trying to make sense of them here.

Looking at Mary's jupyter [notebook](https://github.com/mfisher5/mf-fish546-PCod/blob/master/notebooks/Lanes%201%20and%202%20combined%20pipeline.ipynb) on this topic for guidance, and Eleni's Evernote page about filtering.

#### [1] Remove heading from catalog SNPs file

I think Mary does this manually. Going to look at the file and see if I can sort out why... looks like it's just that commented out line. I wrote a short script to remove the first n lines, and removed the first.

# cstacks version 1.35; catalog generated on 2017-02-14 10:51:19
0	100	1	42	E	0	T	A	-	-
0	100	1	126	E	0	T	A	-	-
0	100	1	127	E	0	T	A	-	-
0	100	1	128	E	0	A	T	-	-
0	100	1	129	E	0	T	A	-	-
0	100	1	130	E	0	T	A	-	-
0	100	2	57	E	0	C	T	-	-
0	100	2	127	E	0	T	C	-	-
0	100	3	10	E	0	T	C	-	-


In [64]:
!python remove_first_n_lines_textfile.py ../../../../Data_for_Analysis/WorkingFolder/pstacks/batch_100.catalog.snps.tsv 1 ../../../../Data_for_Analysis/WorkingFolder/pstacks/batch_100_woheader.catalog.snps.tsv


Your text file has 230573 lines, and you want to extract lines 1 and write it to the file with name ../../../../Data_for_Analysis/WorkingFolder/pstacks/batch_100_woheader.catalog.snps.tsv


In [53]:
cd Data_for_Analysis/WorkingFolder/pstacks/

/mnt/hgfs/Data_for_Analysis/WorkingFolder/pstacks


In [55]:
!head batch_100_woheader.catalog.snps.tsv

0	100	1	42	E	0	T	A	-	-

0	100	1	126	E	0	T	A	-	-

0	100	1	127	E	0	T	A	-	-

0	100	1	128	E	0	A	T	-	-

0	100	1	129	E	0	T	A	-	-



#### [2] add a "_" between Cat and ID in the batch_1 haplotypes.tsv file

I think Mary did this manually again. So first unzip, then manually open in a text editor, and add an underscore.

In [17]:
!gzip -d batch_100.haplotypes.tsv

gzip: batch_100.haplotypes.tsv: unknown suffix -- ignored


Double-check that the haplotypes file looks good. It does!

In [56]:
!head batch_100.haplotypes.tsv

Catalog_ID	Cnt	FG001	FG002	FG003	FG004	FG005	FG006	FG007	FG009	FG010	FG011	FG013	FG014	FG015	FG016	FG017	FG018	FG019	FG021	FG024	FG026	FG027	FG031	FG032	FG033	FG034	FG035	FG100_B	FG101_A	FG101_B	FG102_A	FG102_B	FG103_A	FG103_B	FG104_A	FG104_B	FG106	FG107	FG108	FG109	FG111	FG201	FG202	FG203	FG204	FG205	FG206	FG207	FG209	FG210	FG211	FG212	Q314	Q316	Q318	Q323	Q324	Q325	Q326	Q330	Q332	Q334	Q338	Q339	Q347	Q351	Q352	Q353	Q354	Q355	Q356
119	70	CGTACAGT/CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT/TGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGG/CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CATATAGT/CGTATAGG	CGTATAGT	CGTATAGT/TGTATAGT	CGTATAGT	CGTATAGT	CGTATACT/CGTATAGT	CGTATAGT	CGTATAGT	CGTATAGT	CGCATTGT/CGTATAGT	CGTATAGT	CGTATAGT	C

#### [3] Run Marine's script to make biallelic catalog reference.

Here's a [link to the script](https://github.com/nclowell/RAD_Scallops/blob/master/CRAGIG_run1/Scripts/preparing_file_for_correcting_genotypes.py.txt). 

In [62]:
cd /mnt/hgfs/Ubuntu_Bioinformatics/Git_repo/CRAGIG_run1/Scripts

/mnt/hgfs/Ubuntu_Bioinformatics/Git_repo/CRAGIG_run1/Scripts


In [65]:
!python preparing_file_for_correcting_genotypes.py \
../../../../Data_for_Analysis/WorkingFolder/pstacks/batch_100.haplotypes.tsv \
../../../../Data_for_Analysis/WorkingFolder/pstacks/batch_100_biallelic.catalog.tsv \
../../../../Data_for_Analysis/WorkingFolder/pstacks/batch_100_woheader.catalog.snps.tsv \
1

119 70 CGTACAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT TGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGG CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CATATAGT CGTATAGG CGTATAGT CGTATAGT TGTATAGT CGTATAGT CGTATAGT CGTATACT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGCATTGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTACAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTACAGT CGTATAGT CGTATAGT CGTGTAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT CGTATAGT
182 70 CCACTCCGGGGCCGGCGAGC CCACTCCGGGGCCGGCGAGC TCACTACGGGGCCGGCGAGC CCACTCCGGGGCCGGCGAGC CCACTCCGGGGCCGGCGAGC CCACTCCGGGGCCGGCGAGC CCACTCCGAATCCGGCGAGC CCATTCCGGGGCCGGCGAGC CCACTCCGGGGCCGGCGAGC CCACTCCGGGGCCGGCGAGC CCACTCCGGGGCCGGCGTTC CCACTCCGGGGCCGGCGAGC CCACTCCGGGGCCG

#### [4] create a bash script that will unzip all of the individual .tags.tsv files, and then call Marine's genotypes_verif python script.